<a href="https://colab.research.google.com/github/SarahiGomezM/Actividad-4-Regresi-n-Log-stica-/blob/main/Actividad_4_(Regresi%C3%B3n_Log%C3%ADstica)Mexico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

#Cargamos librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

In [4]:
#Carga desde un archivo .csv sin indice
df = pd.read_csv('Base_Mexico_LIMPIA.csv')
df.head(5)

,Unnamed: 0,id,scrape_id,host_id,latitude,longitude,listing_url,last_scraped,source,name,...,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,reviews_per_month
0,0,898633,20240627045222,4796027,19.42063,-99.16586,https://www.airbnb.com/rooms/898633,2024-06-28,city scrape,City SkyLine View from 1 Bedroom Apt in CDMX,...,4.95,4.93,4.95,4.93,4.96,4.93,4.91,2.0,1.0,0.41
1,1,35797,20240627045222,153786,19.38283,-99.27178,https://www.airbnb.com/rooms/35797,2024-06-27,city scrape,Villa Dante,...,4.80,4.90,4.80,4.90,4.90,4.90,4.80,1.0,0.0,1.50
2,2,44616,20240627045222,196253,19.41162,-99.17794,https://www.airbnb.com/rooms/44616,2024-06-28,city scrape,CONDESA HAUS B&B,...,4.58,4.56,4.70,4.87,4.78,4.98,4.48,4.0,2.0,0.42
3,3,913084,20240627045222,4647293,19.43956,-99.17263,https://www.airbnb.com/rooms/913084,2024-06-28,city scrape,15 blocks from Paseo de la Reforma,...,4.75,4.83,4.72,4.84,4.93,4.90,4.75,1.0,0.0,1.19
4,4,67703,20240627045222,334451,19.41152,-99.16857,https://www.airbnb.com/rooms/67703,2024-06-28,city scrape,"2 bedroom apt. deco bldg, Condesa",...,4.90,4.81,4.75,4.94,4.92,4.98,4.91,2.0,0.0,0.32


In [5]:
#Sustituir valores nulos por un string en  concreto
df["host_response_rate"] =df["host_response_rate"].replace('No captured',0)
# Elimina cualquier símbolo no numérico (por ejemplo, '%') y convierte a tipo float
df['host_response_rate'] = df['host_response_rate'].str.replace('%', '').astype(float)

#Sustituir valores nulos por un string en  concreto
df["host_acceptance_rate"] =df["host_acceptance_rate"].replace('No captured',0)
# Elimina cualquier símbolo no numérico (por ejemplo, '%') y convierte a tipo float
df['host_acceptance_rate'] = df['host_acceptance_rate'].str.replace('%', '').astype(float)

#Realizamos una copia del dataframe
data=df.copy()

#Sustituir valores nulos con promedio o media
df["host_response_rate"]=df["host_response_rate"].fillna(round(data["host_response_rate"].mean(),1))
df["host_acceptance_rate"]=df["host_acceptance_rate"].fillna(round(data["host_acceptance_rate"].mean(),1))

In [6]:
#Rellenamos nulos
df =df.fillna(method="bfill")
df =df.fillna(method="ffill")

<ipython-input-6-eecc153cd203>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df =df.fillna(method="bfill")
<ipython-input-6-eecc153cd203>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df =df.fillna(method="ffill")


In [7]:
nulos=df.isnull().sum().sum()
nulos

0

In [8]:
#ajustar max de filas
pd.options.display.max_rows=10
cuantitativas=df.iloc[ : , 40:71]
cuantitativas

,host_listings_count,host_total_listings_count,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,maximum_nights,minimum_minimum_nights,...,number_of_reviews_l30d,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms
0,3.0,6.0,2.0,1.0,1.0,2.0,1652.0,3.0,48,3.0,...,0.0,4.95,4.93,4.95,4.93,4.96,4.93,4.91,2.0,1.0
1,1.0,1.0,2.0,1.0,1.0,1.0,1057.7,1.0,7,1.0,...,0.0,4.80,4.90,4.80,4.90,4.90,4.90,4.80,1.0,0.0
2,17.0,17.0,2.9,1.4,1.4,1.5,1057.7,1.0,365,1.0,...,0.0,4.58,4.56,4.70,4.87,4.78,4.98,4.48,4.0,2.0
3,1.0,4.0,4.0,1.0,2.0,3.0,814.0,2.0,1125,2.0,...,0.0,4.75,4.83,4.72,4.84,4.93,4.90,4.75,1.0,0.0
4,3.0,4.0,4.0,1.0,2.0,2.0,1923.0,2.0,30,2.0,...,0.0,4.90,4.81,4.75,4.94,4.92,4.98,4.91,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26531,13.0,15.0,2.9,1.4,1.4,1.5,1057.7,2.0,365,2.0,...,0.0,4.80,4.90,4.80,4.90,4.90,4.90,4.80,8.0,0.0
26532,33.0,7.1,3.0,1.0,1.0,2.0,1478.0,1.0,365,1.0,...,0.0,4.80,4.90,4.80,4.90,4.90,4.90,4.80,11.0,0.0
26533,12.0,15.0,3.0,1.0,1.0,1.0,936.0,2.0,365,2.0,...,0.0,4.80,4.90,4.80,4.90,4.90,4.90,4.80,1.0,0.0
26534,3.0,3.0,4.0,1.5,1.0,2.0,768.0,1.0,365,1.0,...,0.0,4.80,4.90,4.80,4.90,4.90,4.90,4.80,3.0,0.0


In [9]:
variables=df.iloc[ : , [19,20,41,42,71,46]]
variables

,host_response_rate,host_acceptance_rate,host_total_listings_count,accommodates,reviews_per_month,price
0,100.0,88.0,6.0,2.0,0.41,1652.0
1,0.0,89.8,1.0,2.0,1.50,1057.7
2,100.0,92.0,17.0,2.9,0.42,1057.7
3,100.0,96.0,4.0,4.0,1.19,814.0
4,89.0,62.0,4.0,4.0,0.32,1923.0
...,...,...,...,...,...,...
26531,100.0,100.0,15.0,2.9,1.50,1057.7
26532,97.0,91.0,7.1,3.0,1.50,1478.0
26533,100.0,100.0,15.0,3.0,1.50,936.0
26534,100.0,100.0,3.0,4.0,1.50,768.0


PRICE

In [10]:
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max=df['price'].max()
Min=df['price'].min()
Limites= [Min, Max]
Limites

[105.0, 2980.0]

In [11]:
#Categorización de variables
#Declaramos 2 intervalos
intervalos=np.linspace(107.0, 2981.0, 3)
intervalos

array([ 107., 1544., 2981.])

In [12]:
#Creamos las categorías
categorias= ["Min.Price", "Max.Price"]

In [13]:
#Finalmente creamos las categorías en la columna numérica
df['price']=pd.cut(x= df['price'], bins=intervalos, labels= categorias )
df['price']

,price
0,Max.Price
1,Min.Price
2,Min.Price
3,Min.Price
4,Max.Price
...,...
26531,Min.Price
26532,Min.Price
26533,Min.Price
26534,Min.Price


In [14]:
df['price'].isnull().sum()

1

In [15]:
df=df.dropna(subset=['price'])

In [16]:
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df[['bathrooms', 'bedrooms', 'beds']]
Var_Dep= df['price']

#Redefinimos las variables
X= Vars_Indep
y= Var_Dep

In [17]:
#Dividimos el conjunto de datos en la parte de entretenimiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=None)

#Se escalan todos los datos
escalar= StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos fit_tranform
X_train= escalar.fit_transform(X_train)
X_test=escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [18]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['Min.Price', 'Min.Price', 'Min.Price', ..., 'Min.Price',
       'Min.Price', 'Min.Price'], dtype=object)

In [19]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión')
print(matriz)

Matriz de Confusión
[[  47  811]
 [  53 4396]]


In [20]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="Min.Price")
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.8442481275206453


In [21]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.8371961560203505


In [22]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Min.Price")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.9880872106091256


Reviews_per_month

In [23]:
df['reviews_per_month'].isnull().sum()

0

In [24]:
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max=df['reviews_per_month'].max()
Min=df['reviews_per_month'].min()
Limites= [Min, Max]
Limites

[0.01, 5.39]

In [25]:
#Categorización de variables
#Declaramos 2 intervalos
intervalos=np.linspace(0.02, 5.40, 3)
intervalos

array([0.02, 2.71, 5.4 ])

In [26]:
#Creamos las categorías
categorias= ["Min.Rewiews", "Max.Rewiews"]

#Finalmente creamos las categorías en la columna numérica
df['reviews_per_month']=pd.cut(x= df['reviews_per_month'], bins=intervalos, labels= categorias )
df['reviews_per_month']

,reviews_per_month
0,Min.Rewiews
1,Min.Rewiews
2,Min.Rewiews
3,Min.Rewiews
4,Min.Rewiews
...,...
26531,Min.Rewiews
26532,Min.Rewiews
26533,Min.Rewiews
26534,Min.Rewiews


In [27]:
df['reviews_per_month'].isnull().sum()

226

In [28]:
df=df.dropna(subset=['reviews_per_month'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26309 entries, 0 to 26535
Data columns (total 72 columns):
 #   Column                                        Non-Null Count  Dtype   
---  ------                                        --------------  -----   
 0   Unnamed: 0                                    26309 non-null  int64   
 1   id                                            26309 non-null  int64   
 2   scrape_id                                     26309 non-null  int64   
 3   host_id                                       26309 non-null  int64   
 4   latitude                                      26309 non-null  float64 
 5   longitude                                     26309 non-null  float64 
 6   listing_url                                   26309 non-null  object  
 7   last_scraped                                  26309 non-null  object  
 8   source                                        26309 non-null  object  
 9   name                                          26309 non

In [29]:
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df[['review_scores_cleanliness', 'review_scores_communication', 'review_scores_location', 'review_scores_value']]
Var_Dep= df['reviews_per_month']
#Redefinimos las variables
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entretenimiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=None)

#Se escalan todos los datos
escalar= StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos fit_tranform
X_train= escalar.fit_transform(X_train)
X_test=escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [30]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['Min.Rewiews', 'Min.Rewiews', 'Min.Rewiews', ..., 'Min.Rewiews',
       'Min.Rewiews', 'Min.Rewiews'], dtype=object)

In [31]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión')
print(matriz)

Matriz de Confusión
[[   0  780]
 [   0 4482]]


In [32]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="Min.Rewiews")
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.8517673888255416


In [33]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.8517673888255416


In [34]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Min.Rewiews")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
1.0


PRICE

In [35]:
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max=df['price'].max()
Min=df['price'].min()
Limites= [Min, Max]
Limites

['Min.Price', 'Max.Price']

In [36]:
#Categorización de variables
#Declaramos 2 intervalos
intervalos=np.linspace(104.0, 2981.0, 3)
intervalos

array([ 104. , 1542.5, 2981. ])

In [37]:
#Creamos las categorías
categorias= ["Min.Price", "Max.Price"]

In [38]:
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df[['minimum_nights', 'maximum_nights']]
Var_Dep= df['price']
#Redefinimos las variables
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entretenimiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=None)

#Se escalan todos los datos
escalar= StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos fit_tranform
X_train= escalar.fit_transform(X_train)
X_test=escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [39]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['Min.Price', 'Min.Price', 'Min.Price', ..., 'Min.Price',
       'Min.Price', 'Min.Price'], dtype=object)

In [40]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión')
print(matriz)

Matriz de Confusión
[[   0  874]
 [   0 4388]]


In [41]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="Min.Price")
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.8339034587609274


In [42]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.8339034587609274


In [43]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Min.Price")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
1.0


Availability_365

In [44]:
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max=df['availability_365'].max()
Min=df['availability_365'].min()
Limites= [Min, Max]
Limites

[0, 365]

In [45]:
#Categorización de variables
#Declaramos 2 intervalos
intervalos=np.linspace(0, 365, 3)
intervalos

array([  0. , 182.5, 365. ])

In [46]:
#Creamos las categorías
categorias= ["Inicio_Año", "Fin_Año"]

#Finalmente creamos las categorías en la columna numérica
df['availability_365']=pd.cut(x= df['availability_365'], bins=intervalos, labels= categorias )
df['availability_365']

,availability_365
0,Fin_Año
1,Fin_Año
2,Fin_Año
3,Fin_Año
4,Fin_Año
...,...
26531,Fin_Año
26532,Fin_Año
26533,Fin_Año
26534,Fin_Año


In [47]:
df=df.dropna(subset=['availability_365'])

In [48]:
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df[['minimum_nights', 'maximum_nights']]
Var_Dep= df['availability_365']
#Redefinimos las variables
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entretenimiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=None)

#Se escalan todos los datos
escalar= StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos fit_tranform
X_train= escalar.fit_transform(X_train)
X_test=escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [49]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['Fin_Año', 'Fin_Año', 'Fin_Año', ..., 'Fin_Año', 'Fin_Año',
       'Fin_Año'], dtype=object)

In [50]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión')
print(matriz)

Matriz de Confusión
[[3048  179]
 [1468  148]]


In [51]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="Fin_Año")
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.674933569530558


In [52]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.6599215362378691


In [53]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Fin_Año")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.9445305237062287


host_total_listings_count

In [54]:
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max=df['host_total_listings_count'].max()
Min=df['host_total_listings_count'].min()
Limites= [Min, Max]
Limites

[1.0, 39.0]

In [55]:
#Categorización de variables
#Declaramos 2 intervalos
intervalos=np.linspace(2.0, 40, 3)
intervalos

array([ 2., 21., 40.])

In [56]:
#Creamos las categorías
categorias= ["Min.Count", "Max.Count"]

#Finalmente creamos las categorías en la columna numérica
df['host_total_listings_count']=pd.cut(x= df['host_total_listings_count'], bins=intervalos, labels= categorias )
df['host_total_listings_count']

,host_total_listings_count
0,Min.Count
1,NaN
2,Min.Count
3,Min.Count
4,Min.Count
...,...
26531,Min.Count
26532,Min.Count
26533,Min.Count
26534,Min.Count


In [57]:
df=df.dropna(subset=['host_total_listings_count'])

In [58]:
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df[['latitude', 'longitude', 'availability_30']]
Var_Dep= df['host_total_listings_count']
#Redefinimos las variables
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entretenimiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=None)

#Se escalan todos los datos
escalar= StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos fit_tranform
X_train= escalar.fit_transform(X_train)
X_test=escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [59]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['Min.Count', 'Min.Count', 'Min.Count', ..., 'Min.Count',
       'Min.Count', 'Min.Count'], dtype=object)

In [60]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión')
print(matriz)

Matriz de Confusión
[[   0  350]
 [   0 3043]]


In [61]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="Min.Count")
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.8968464485705865


In [62]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.8968464485705865


In [63]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Min.Count")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
1.0


*Variables Categóricas*

room_type =beds , bedrooms ,

In [64]:
# Reemplazar los valores en la columna 'room_type'
df['room_type'] = df['room_type'].replace(["Entire home/apt"], "Home type")
df['room_type'] = df['room_type'].replace(["Private room", "Hotel room", "Shared room"], "Room type")

# Redefinimos las variables independientes (X) y dependiente (y)
Vars_Indep = df[['bathrooms', 'bedrooms', 'beds']]
Var_Dep = df['room_type']  # Asegúrate de usar 'room_type' con minúsculas

# Asignamos las variables
X = Vars_Indep
y = Var_Dep

# Dividimos el conjunto de datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=None)

# Se escalan los datos
escalar = StandardScaler()

# Aplicamos el escalado a los datos de entrenamiento y de prueba
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# Definimos el algoritmo a utilizar: Regresión Logística
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

# Entrenamos el modelo con los datos de entrenamiento
algoritmo.fit(X_train, y_train)

# Realizamos la predicción sobre los datos de prueba
y_pred = algoritmo.predict(X_test)

# Verificamos la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

# Calculamos la precisión del modelo (ajusta pos_label para que coincida con tus clases)
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average="binary", pos_label="Home type")  # o "Room type"
print('Precisión del modelo:')
print(precision)

# Calculamos la exactitud (accuracy) del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

# Calculamos la sensibilidad del modelo (ajusta pos_label también)
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Home type")  # o "Room type"
print('Sensibilidad del modelo:')
print(sensibilidad)

Matriz de Confusión:
[[2104  184]
 [ 871  234]]
Precisión del modelo:
0.7072268907563025
Exactitud del modelo:
0.6890657235484822
Sensibilidad del modelo:
0.9195804195804196


In [65]:
# Reemplazar los valores en la columna 'room_type'
df['room_type'] = df['room_type'].replace(["Entire home/apt"], "Home type")
df['room_type'] = df['room_type'].replace(["Private room", "Hotel room", "Shared room"], "Room type")

# Redefinimos las variables independientes (X) y dependiente (y)
Vars_Indep = df[['bathrooms', 'bedrooms', 'beds']]
Var_Dep = df['room_type']  # Asegúrate de usar 'room_type' con minúsculas

# Asignamos las variables
X = Vars_Indep
y = Var_Dep

# Dividimos el conjunto de datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=None)

# Se escalan los datos
escalar = StandardScaler()

# Aplicamos el escalado a los datos de entrenamiento y de prueba
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# Definimos el algoritmo a utilizar: Regresión Logística
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

# Entrenamos el modelo con los datos de entrenamiento
algoritmo.fit(X_train, y_train)

# Realizamos la predicción sobre los datos de prueba
y_pred = algoritmo.predict(X_test)

# Verificamos la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

# Calculamos la precisión del modelo (ajusta pos_label para que coincida con tus clases)
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average="binary", pos_label="Home type")  # o "Room type"
print('Precisión del modelo:')
print(precision)

# Calculamos la exactitud (accuracy) del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

# Calculamos la sensibilidad del modelo (ajusta pos_label también)
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Home type")  # o "Room type"
print('Sensibilidad del modelo:')
print(sensibilidad)


Matriz de Confusión:
[[2123  179]
 [ 856  235]]
Precisión del modelo:
0.712655253440752
Exactitud del modelo:
0.6949602122015915
Sensibilidad del modelo:
0.922241529105126


In [68]:


# Reemplazar los valores en la columna 'instant_bookable'

# Redefinimos las variables independientes (X) y dependiente (y)
Vars_Indep = df[['minimum_nights', 'maximum_nights','availability_365']]
Var_Dep = df['instant_bookable']  # Asegúrate de usar 'room_type' con minúsculas

# Asignamos las variables
X = Vars_Indep
y = Var_Dep

# Identify columns with non-numeric values
for col in X.columns:
  if not pd.api.types.is_numeric_dtype(X[col]):
    print(f"Column '{col}' has non-numeric values:")
    print(X[col].value_counts())
    # Handle non-numeric values (e.g., replace with a numeric value, drop the column, etc.)
    # Example: Replace non-numeric values with 0
    X[col] = pd.to_numeric(X[col], errors='coerce').fillna(0)

# Dividimos el conjunto de datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=None)

# Se escalan los datos
escalar = StandardScaler()

# Aplicamos el escalado a los datos de entrenamiento y de prueba
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# Definimos el algoritmo a utilizar: Regresión Logística
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

# Entrenamos el modelo con los datos de entrenamiento
algoritmo.fit(X_train, y_train)

# Realizamos la predicción sobre los datos de prueba
y_pred = algoritmo.predict(X_test)

# Verificamos la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

# Calculamos la precisión del modelo (ajusta pos_label para que coincida con tus clases)
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average="binary", pos_label="t")  # o "Room type"
print('Precisión del modelo:')
print(precision)

# Calculamos la exactitud (accuracy) del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

# Calculamos la sensibilidad del modelo (ajusta pos_label también)
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="t")  # o "Room type"
print('Sensibilidad del modelo:')
print(sensibilidad)

Column 'availability_365' has non-numeric values:
availability_365
Fin_Año       11630
Inicio_Año     5333
Name: count, dtype: int64
Matriz de Confusión:
[[ 914  782]
 [ 511 1186]]


<ipython-input-68-3fe865c0290f>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = pd.to_numeric(X[col], errors='coerce').fillna(0)


Precisión del modelo:
0.6026422764227642
Exactitud del modelo:
0.618921308576481
Sensibilidad del modelo:
0.6988803771361226


In [70]:


#Variable dicotómica (host_response_time)

df['host_response_time']= df['host_response_time'].replace(["within an hour","within a few hours","within a day","a few days or more"], "Fast response")

# Redefinimos las variables independientes (X) y dependiente (y)
Vars_Indep = df[['review_scores_communication', 'reviews_per_month']]
Var_Dep = df['host_response_time']
# Asignamos las variables
X = Vars_Indep
y = Var_Dep

# Identify columns with non-numeric values
for col in X.columns:
  if not pd.api.types.is_numeric_dtype(X[col]):
    print(f"Column '{col}' has non-numeric values:")
    print(X[col].value_counts())
    # Handle non-numeric values (e.g., replace with a numeric value, drop the column, etc.)
    # Example: Replace non-numeric values with 0
    X[col] = pd.to_numeric(X[col], errors='coerce').fillna(0)

# Dividimos el conjunto de datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=None)

# Se escalan los datos
escalar = StandardScaler()

# Aplicamos el escalado a los datos de entrenamiento y de prueba
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# Definimos el algoritmo a utilizar: Regresión Logística
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

# Entrenamos el modelo con los datos de entrenamiento
algoritmo.fit(X_train, y_train)

# Realizamos la predicción sobre los datos de prueba
y_pred = algoritmo.predict(X_test)

# Verificamos la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

# Calculamos la precisión del modelo (ajusta pos_label para que coincida con tus clases)
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average="binary", pos_label="Fast response")  # o "Room type"
print('Precisión del modelo:')
print(precision)

# Calculamos la exactitud (accuracy) del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

# Calculamos la sensibilidad del modelo (ajusta pos_label también)
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Fast response")  # o "Room type"
print('Sensibilidad del modelo:')
print(sensibilidad)

Column 'reviews_per_month' has non-numeric values:
reviews_per_month
Min.Rewiews    14033
Max.Rewiews     2930
Name: count, dtype: int64
Matriz de Confusión:
[[3173    0]
 [ 220    0]]


<ipython-input-70-80c7ff607c21>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = pd.to_numeric(X[col], errors='coerce').fillna(0)


Precisión del modelo:
0.9351606248157972
Exactitud del modelo:
0.9351606248157972
Sensibilidad del modelo:
1.0


In [72]:
#Variable dicotómica (host_response_time)


df = df[df['host_is_superhost'] != 'No captured']
df['host_is_superhost']=df['host_is_superhost'].replace(['t'],'True')
df['host_is_superhost']=df['host_is_superhost'].replace(['f'],'False')
# Redefinimos las variables independientes (X) y dependiente (y)
Vars_Indep = df[['availability_30', 'host_total_listings_count','review_scores_rating']]
Var_Dep = df['host_is_superhost']
# Asignamos las variables
X = Vars_Indep
y = Var_Dep

# Identify columns with non-numeric values
for col in X.columns:
  if not pd.api.types.is_numeric_dtype(X[col]):
    print(f"Column '{col}' has non-numeric values:")
    print(X[col].value_counts())
    # Handle non-numeric values (e.g., replace with a numeric value, drop the column, etc.)
    # Example: Replace non-numeric values with 0
    X[col] = pd.to_numeric(X[col], errors='coerce').fillna(0)

# Dividimos el conjunto de datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=None)

# Se escalan los datos
escalar = StandardScaler()

# Aplicamos el escalado a los datos de entrenamiento y de prueba
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# Definimos el algoritmo a utilizar: Regresión Logística
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

# Entrenamos el modelo con los datos de entrenamiento
algoritmo.fit(X_train, y_train)

# Realizamos la predicción sobre los datos de prueba
y_pred = algoritmo.predict(X_test)

# Verificamos la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

# Calculamos la precisión del modelo (ajusta pos_label para que coincida con tus clases)
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average="binary", pos_label="False")  # o "Room type"
print('Precisión del modelo:')
print(precision)

# Calculamos la exactitud (accuracy) del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

# Calculamos la sensibilidad del modelo (ajusta pos_label también)
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="False")  # o "Room type"
print('Sensibilidad del modelo:')
print(sensibilidad)

Column 'host_total_listings_count' has non-numeric values:
host_total_listings_count
Min.Count    14060
Max.Count     1466
Name: count, dtype: int64
Matriz de Confusión:
[[1096  523]
 [ 661  826]]


<ipython-input-72-2ad9639643d1>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = pd.to_numeric(X[col], errors='coerce').fillna(0)


Precisión del modelo:
0.6237905520774046
Exactitud del modelo:
0.6188023180940115
Sensibilidad del modelo:
0.6769610870907968


In [73]:
#Variable dicotómica (host_has_profile_pic)

df = df[df['host_has_profile_pic'] != 'No captured']
df['host_has_profile_pic']=df['host_has_profile_pic'].replace(['t'],'True')
df['host_has_profile_pic']=df['host_has_profile_pic'].replace(['f'],'False')

# Redefinimos las variables independientes (X) y dependiente (y)
Vars_Indep = df[['number_of_reviews','minimum_nights','maximum_nights']]
Var_Dep = df['host_has_profile_pic']
# Asignamos las variables
X = Vars_Indep
y = Var_Dep

# Dividimos el conjunto de datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=None)

# Se escalan los datos
escalar = StandardScaler()

# Aplicamos el escalado a los datos de entrenamiento y de prueba
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# Definimos el algoritmo a utilizar: Regresión Logística
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

# Entrenamos el modelo con los datos de entrenamiento
algoritmo.fit(X_train, y_train)

# Realizamos la predicción sobre los datos de prueba
y_pred = algoritmo.predict(X_test)

# Verificamos la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

# Calculamos la precisión del modelo (ajusta pos_label para que coincida con tus clases)
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average="binary", pos_label="True")  # o "Room type"
print('Precisión del modelo:')
print(precision)

# Calculamos la exactitud (accuracy) del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

# Calculamos la sensibilidad del modelo (ajusta pos_label también)
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="True")  # o "Room type"
print('Sensibilidad del modelo:')
print(sensibilidad)

Matriz de Confusión:
[[   0   18]
 [   0 3088]]
Precisión del modelo:
0.9942047649710238
Exactitud del modelo:
0.9942047649710238
Sensibilidad del modelo:
1.0
